## Code written and working on simulated data set

In [83]:
import numpy as np
import scipy as sp

In [117]:
def PhiGamma(alpha, beta, w, T):
    k = len(alpha)
    N = len(w)
    alpha = np.array(alpha)
    beta = np.array(beta)
    gamma = np.tile(alpha + N/k, ((T+1),1))
    phi = np.array([[[1/k]*k]*N]*(T+1))
    for t in range(T):
        for n in range(N):
            for i in range(k):
                phi[t+1,n,i] = beta[i,n] * np.exp(sp.special.psi(gamma[t,i])) #beta[i,n], n should be w_n.
            phi_const = np.sum(phi[t+1,n,])
            phi[t+1,n,] = phi[t+1,n,]/phi_const
        gamma[t+1,] = alpha + np.sum(phi[t+1,], axis=0)
    return(gamma[T,],phi[T,])

In [121]:
#Test function
N=4
T=5
alpha = np.array([1/3,1/3,1/3])
beta = np.array([[1/2,1/2,1/2,1/2,1/2,1/2], [1/2,1/2,1/2,1/2,1/2,1/2], [1/2,1/2,1/2,1/2,1/2,1/2]])
w=0
a = np.stack((beta,beta))
PhiGamma(alpha, beta, w, N, T)

(array([ 1.66666667,  1.66666667,  1.66666667]),
 array([[ 0.33333333,  0.33333333,  0.33333333],
        [ 0.33333333,  0.33333333,  0.33333333],
        [ 0.33333333,  0.33333333,  0.33333333],
        [ 0.33333333,  0.33333333,  0.33333333]]))

## Background written

## Flesh out sections and subsections as appropriate